In [1]:
from astropy.io import fits
from astropy import wcs
import bz2
import os
import urllib.request
import numpy as np
import copy
import matplotlib.pyplot as plt
import csv
from astropy.utils.data import get_pkg_data_filename
from matplotlib.colors import LogNorm
from scipy.interpolate import interp1d

In [52]:
def convert_pix_to_petrorad(resultscsv, outputname):
    with open(resultscsv) as csvfile:
            reader = csv.reader(csvfile)
            filenamelist=[]
            resultslist=[]
            petroradlist=[]
            newresultslist=[]
            #Gets data on everything except 1st row
            colnames=next(reader)
            xpixels=[float(i) for i in colnames[2:]]
            xpetrorad=np.arange(0.05,1.3,0.01)
            for row in reader:
                filenamelist.append(row[0])
                petroradlist.append(row[1])
                resultslist.append(row[2:])
            for i in range(len(filenamelist)):
                #create x and y to use in interpolation
                xint=[]
                yint=[]
                for j in range(len(resultslist[i])):
                    if not resultslist[i][j]=='nan':
                        xint.append(float(xpixels[j]))
                        yint.append(float(resultslist[i][j]))
                    else:
                        break
                f=interp1d(xint,yint, kind='linear', fill_value='extrapolate')
                xpetroradpix=[float(petroradlist[i])*j for j in xpetrorad]
                ypetrorad=[f(k) for k in xpetroradpix]
                newresultslist.append(ypetrorad)
            with open(outputname,'w+') as outputfile:
                writer=csv.writer(outputfile)
                writer.writerow(['Filename']+[str(k) for k in xpetrorad])
                for i in range(len(filenamelist)):
                        row=[filenamelist[i]]+newresultslist[i]
                        writer.writerow(row)

In [55]:
convert_pix_to_petrorad('victor results down 1 pix', 'victor_results_fraction_petrorad.csv')